In [ ]:
# This is a CNN build for CIFAR-10 data set and is very to similar to the CNN for MNIST data, 
# the only difference is it is not reshaped before input to the first layer.
# This network has 2 convolutional layers, two ReLu layers and three FC layers 
# The architecture of the CNN is shown below

'''
•	Convolution Layer 1:
    o	Convolution Shape: [batch,H,W,depth]= [1, 32, 32, 32]
    o	2D datasize: 32X32
    o	Input Shape: [32X32X 3] 
    o	Output Shape: [32X32X 32] 
    o	Filter/Kernel/Window size: 5X5
    o	Filter Shape=  weight shape: [5,5,3,32]
    o	Stride =1
    o	Stride Shape: [1,1,1,1]
    o	Padding: SAME
    o	Activation Function Layer: ReLU
•	ReLu Layer 1:
    o	No filter (only process data, no extract/learn features)
    o	Input Shape: [32X32X32]
    o	Output Shape: [32X32X32]
•	Pooling Layer 1:
    o	Pooling  method: Max pooling
    o	Filter/Kernal/Window size: 2X2
    o	Filter/Kernal/Window Shape: [1,2,2,1]
    o	Stride: 2
    o	Stride Shape: [1,2,2,1]
    o	Padding: SAME
    o	Input channel: 32 inputs
    o	Input shape: [32X32X 32]
    o	Output channel: 32 outputs
    o	Output shape: 16X16X32
•	Convolution Layer 2:
    o	Convolution Shape: [batch,H,W,depth]=[1,16,16,64]
    o	2D datasize: 16X16
    o	Input Shape: 16X16X32 (depth=1 in channel)
    o	Output Shape: 14X14X64 (depth=32 out channel)
    o	Filter/Kernel/Window size: 5X5
    o	Filter Shape=  weight shape: [5,5,32,64]
    o	Stride =1
    o	Stride Shape: [1,1,1,1]
    o	Padding: SAME
    o	Activation Function Layer: ReLu
•	ReLu Layer 2:
    o	No filter (only process data, no extract/learn features)
    o	Input Shape: 14X14X64
    o	Output Shape: 14X14X64
•	Pooling Layer 2:
    o	Pooling  method: Max pooling
    o	Filter/Kernal/Window size: 2X2
    o	Filter/Kernal/Window Shape: [1,2,2,1]
    o	Stride: 2
    o	Stride Shape: [1,2,2,1]
    o	Padding: SAME
    o	Input channel: 64 inputs
    o	Input shape: 16X16X64
    o	Output channel: 64 outputs
    o	Output shape: 8X8X64
•	Fully Connected Layer 1(FC_1):
    o	FC_1 shape: [inputs (in_channels), outputs (output_channels)] = [8X8X64X1024]
•	Fully Connected Layer 2(FC_2):
    o	FC_2 shape: [inputs (in_channels), outputs (output_channels)] = [1024X1024]
•	Fully Connected Layer 3 (FC_3):
    o	FC_3 shape: [inputs (in_channels), outputs (output_channels)] = [1024, 10]

'''



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

CIFAR_DIR = 'JP_NTBK/CIFAR_10_DATA/'

# Load Data

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        cifar_dict = pickle.load(fo, encoding='bytes')
    return cifar_dict

In [3]:
def load_data(data_dir):
    data_batches = []
    labels_batches = []
    for i in range(1, 6):
        batch = unpickle(f"{data_dir}/data_batch_{i}")
        data_batches.append(batch[b'data'])
        labels_batches.append(batch[b'labels'])
    data = np.concatenate(data_batches)
    labels = np.concatenate(labels_batches)
    return data, labels

In [4]:
data_dir = CIFAR_DIR
data, labels = load_data(data_dir)
print("Data:::::::::", data)

Data::::::::: [[ 59  43  50 ... 140  84  72]
 [154 126 105 ... 139 142 144]
 [255 253 253 ...  83  83  84]
 ...
 [ 35  40  42 ...  77  66  50]
 [189 186 185 ... 169 171 171]
 [229 236 234 ... 173 162 161]]


In [5]:
dirs = ['batches.meta','data_batch_1','data_batch_2','data_batch_3','data_batch_4','data_batch_5','test_batch']

all_data = [0,1,2,3,4,5,6]

for i,direc in zip(all_data,dirs):
    all_data[i] = unpickle(CIFAR_DIR+direc)

    
batch_meta = all_data[0]
data_batch1 = all_data[1]
data_batch2 = all_data[2]
data_batch3 = all_data[3]
data_batch4 = all_data[4]
data_batch5 = all_data[5]
test_batch = all_data[6]

In [6]:
def one_hot_encode(vec, vals=10):
    n = len(vec)
    out = np.zeros((n, vals))
    out[range(n), vec] = 1
    return out

# Helper Class


In [7]:
class CifarHelper():
    
    def __init__(self, data_batch1, data_batch2, data_batch3, data_batch4, data_batch5, test_batch):
        self.i = 0
        self.all_train_batches = [data_batch1,data_batch2,data_batch3,data_batch4,data_batch5]
        self.test_batch = [test_batch]
        self.training_images = None
        self.training_labels = None
        self.test_images = None
        self.test_labels = None
    
    def set_up_images(self):
        
        print("Setting Up Training Images and Labels")
        self.training_images = np.vstack([d[b"data"] for d in self.all_train_batches])
        train_len = len(self.training_images)
        
        self.training_images = self.training_images.reshape(train_len,3,32,32).transpose(0,2,3,1)/255
        self.training_labels = one_hot_encode(np.hstack([d[b"labels"] for d in self.all_train_batches]), 10)
        
        print("Setting Up Test Images and Labels")
        self.test_images = np.vstack([d[b"data"] for d in self.test_batch])
        test_len = len(self.test_images)
        
        self.test_images = self.test_images.reshape(test_len,3,32,32).transpose(0,2,3,1)/255
        self.test_labels = one_hot_encode(np.hstack([d[b"labels"] for d in self.test_batch]), 10)

    def next_batch(self, batch_size):
        x = self.training_images[self.i:self.i+batch_size]
        y = self.training_labels[self.i:self.i+batch_size]
        self.i = (self.i + batch_size) % len(self.training_images)
        return x, y



# Building the network 

In [8]:
# This function returns a tf.Variable used to store weights in a filter
# The values are initialized with values that can be used to randomize weights

def init_weights(filter_shape):
    init_random_dist = tf.truncated_normal(filter_shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [9]:
# This function returns a tf.Variable used to store bias terms
# The variable is initialized with values that can be used to initialize biases

def init_bias(bias_shape):
    init_vals = tf.constant(0.1, shape=bias_shape)
    return tf.Variable(init_vals)

In [10]:
# Set Up Convolution Layer and Perform Convolution Computation: Dot Product (X * W)

def create_convolution_layer_and_compute_dot_products(inputs, filter_shape):
    # Initialize the weights = initialize filters
    filters = tf.Variable(tf.random.normal(filter_shape))
    
    # Create a convolution layer
    conv_layer_outputs = tf.nn.conv2d(inputs, filters, strides=[1, 1, 1, 1], padding='SAME')
    return conv_layer_outputs

In [11]:
# setup a Relu layer and perform computation: Dot product + Bias(x.W+b)

def create_convolution_layer_and_compute_dotproducts_plus_b(inputs, filter_shape):
    # Initialize bias for each output channel
    b= init_bias([filter_shape[3]])
    
    # Perform the computation first by adding: inputs (x * W) + b
    relu_layer_outputs = tf.nn.relu(inputs + b)
    return relu_layer_outputs

In [12]:
# Set Up a Pooling Layer and Reduce Spatial Size

# 1) Define Function to Create Pooling Layer
# 2) Then, to reduce the spatial size of the inputs

# Pooling method: Max pooling
# Kernel size: 2 x 2
# Stride: 2

def create_maxpool2by2_and_reduce_Spatial_size(inputs):
    
    # Parameters:
    # inputs: Outputs of the previous convolutional layer
    
    pooling_layer_outputs = tf.nn.max_pool(inputs, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    return pooling_layer_outputs

In [13]:
# Define a function to compute the fully connected layer

def create_fully_connected_layer_and_compute_dotproduct_plus_bias(inputs, output_size):
    input_size = int(inputs.get_shape()[1])
    W=init_weights([input_size, output_size])
    b=init_bias([output_size])
    
    fc_xW_plus_bias_outputs=tf.matmul(inputs,W)+b
    
    return fc_xW_plus_bias_outputs

In [14]:
#Create placeholders for Input and Labels: x and y_true

x=tf.placeholder(tf.float32,shape=[None,32,32,3])
y_true=tf.placeholder(tf.float32,shape=[None,10])


In [15]:
# Create the first convolutional layer, ReLu layer, and perform Computation: x*W+b

conv_layer_1_output= create_convolution_layer_and_compute_dot_products(x, filter_shape=[5, 5, 3, 32])

conv_relu_layer_1_outputs=create_convolution_layer_and_compute_dotproducts_plus_b(conv_layer_1_output, filter_shape=[5,5,3,32])


In [16]:
# Create first pooling layer and reduce size

pooling_layer_1_outputs=create_maxpool2by2_and_reduce_Spatial_size(conv_relu_layer_1_outputs)

In [17]:
# Create second convolutional layer, ReLu layer, and perform Computation: x*W+b

conv_layer_2_output= create_convolution_layer_and_compute_dot_products(pooling_layer_1_outputs, filter_shape=[5, 5, 32, 64])

conv_relu_layer_2_outputs=create_convolution_layer_and_compute_dotproducts_plus_b(conv_layer_2_output, filter_shape=[5,5,32,64])


In [18]:
# Create second pooling layer and reduce size

pooling_layer_2_outputs=create_maxpool2by2_and_reduce_Spatial_size(conv_relu_layer_2_outputs)

In [19]:
# Reshape/flatten data and make it ready to be fed in first FC layer

pooling_layer_2_outputs_flat=tf.reshape(pooling_layer_2_outputs,[-1,8*8*64])

In [20]:
# Create first FC Layer, ReLu layer, and output data to dropout layer

fc_layer_1_outputs=create_fully_connected_layer_and_compute_dotproduct_plus_bias(pooling_layer_2_outputs_flat, output_size=1024)

fc_relu_layer_1_outputs=tf.nn.relu(fc_layer_1_outputs)

In [21]:
# Create a dropout layer and Dropout a fraction of outputs randomly

hold_prob=tf.placeholder(tf.float32)

fc_dropout_outputs=tf.nn.dropout(fc_relu_layer_1_outputs,keep_prob=hold_prob)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [22]:
# Create final FC layer 

y_pred=create_fully_connected_layer_and_compute_dotproduct_plus_bias(fc_dropout_outputs, output_size=10)

In [23]:
# Create Loss Function and Calculate Softmax cross entropy Loss

softmax_cross_entropy_loss=tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred)

# Compute the mean of losses
cross_entropy_mean=tf.reduce_mean(softmax_cross_entropy_loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [24]:
# Create an optimizer to Optimize CNN Model and Set Learning Rate

optimizer=tf.train.AdamOptimizer(learning_rate=0.001)

In [25]:
# Create a trainer to trainig CNN Model

cnn_trainer=optimizer.minimize(cross_entropy_mean)

In [26]:
#### Set up image data: Calling CifarHelper.set_up_images()
#initialize the class CifarHelper

ch = CifarHelper(data_batch1, data_batch2, data_batch3, data_batch4, data_batch5, test_batch)
ch.set_up_images()

Setting Up Training Images and Labels
Setting Up Test Images and Labels


# Training and Testing 

In [27]:
# Train and test CNN deep learning Model on CIFAR-10 dataset

# Create a varaible to initializer to initialize all variables

vars_initializer=tf.global_variables_initializer()

steps=5000

with tf.Session() as sess:
    sess.run(vars_initializer)
    for i in range(steps):
        batch = ch.next_batch(100)
        batch_x=batch[0]
        batch_y=batch[1]

        sess.run(cnn_trainer, feed_dict={x: batch_x, y_true: batch_y, hold_prob: 0.5})

        if i % 100 == 0:
          print('ON STEP: {}'.format(i))
          print('ACCURACY: ')

          matches=tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
          acc=tf.reduce_mean(tf.cast(matches,tf.float32))
          test_accuracy=sess.run(acc, feed_dict={x: ch.test_images, y_true: ch.test_labels, hold_prob: 1.0})

          print(test_accuracy)
          print('\n')



User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   OMP_NUM_THREADS=8

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_HAND_THREAD=false
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_MWAIT_HINTS=0
   KMP_NUM_LOCKS_IN_BLOCK=1

ON STEP: 0
ACCURACY: 
0.1


ON STEP: 100
ACCURACY: 
0.1217


ON STEP: 200
ACCURACY: 
0.1115


ON STEP: 300
ACCURACY: 
0.1051


ON STEP: 400
ACCURACY: 
0.1039


ON STEP: 500
ACCURACY: 
0.1029


ON STEP: 600
ACCURACY: 
0.1029


ON STEP: 700
ACCURACY: 
0.1023


ON STEP: 800
ACCURACY: 
0.1023


ON STEP: 900
ACCURACY: 
0.1026


ON STEP: 1000
ACCURACY: 
0.1023


ON STEP: 1100
ACCURACY: 
0.1015


ON STEP: 1200
ACCURACY: 
0.1017


ON STEP: 1300
ACCURACY: 
0.1019


ON STEP: 1400
ACCURACY: 
0.1019


ON STEP: 1500
ACCURACY: 
0.1021


ON STEP: 1600
ACCURACY: 
0.1026


ON STEP: 1700
ACCURACY: 
0.1024


ON STEP: 1800
ACCURACY: 
0.1022


ON STEP: 1900
ACCURACY: 
0.1018


ON STEP: 2000
ACCURACY: 
0.1019


ON STEP: 2100
ACCURACY: 
0.1016


ON STEP: 2200
ACCURACY: 
0.1012


ON STEP: 2300
ACCURACY: 
0.1019


ON STEP: 2400
ACCURACY: 
0.1016


ON STEP: 2500
ACCURACY: 
0.1012


ON STEP: 2600
ACCURACY: 
0.1011


ON STEP: 2700
ACCURACY: 
0.1013


ON STEP: 2800
ACCURACY: 
0.1016


ON STEP: 2900
ACCURACY: 
0.10